In [0]:
from delta.tables import *

stage_table_name = "pyspark_playground.tracking_orders.stage_zn_new"
target_table_name = "pyspark_playground.tracking_orders.target_zn_new"

# Read data from stage table
stage_df = spark.read.table(stage_table_name)

# Create equivalent target schema if target table doesn't exists

if not spark._jsparkSession.catalog().tableExists(target_table_name):
    stage_df.write.format("delta").saveAsTable(target_table_name)
else:
    # Perform delta table merge query for upsert based on tracking_num column
    target_table = DeltaTable.forName(spark, target_table_name)

    # Define merge condition based on tracking_num column
    merge_condition = "stage.tracking_num = target.tracking_num"

    # Perform merge
    target_table.alias("target")\
        .merge(stage_df.alias("stage"),merge_condition)\
        .whenMatchedDelete()\
        .execute()

    stage_df.write.format("delta").mode("append").saveAsTable(target_table_name)